In [52]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error
import pandas as pd
import mlflow
import os
import numpy as np

In [53]:
"""
Descarga de Archivos desde Google Drive sin Autenticación

Este script permite descargar archivos específicos desde Google Drive directamente a una ubicación local,
utilizando el ID de Google Drive del archivo (el archivo debe ser accesible para cualquier persona que tenga la URL).

El script procesa un archivo de texto ('archivos_info.txt') que contiene las IDs de Google Drive de los archivos,
los nombres con los que se desean guardar localmente, y las rutas locales de almacenamiento, todo separado por comas.

Formato esperado de 'archivos_info.txt':
ID_de_Google_Drive,Nombre_Archivo_Local,Ruta_Local

Ejemplo:
1hGvKmNAkK...,mi_archivo.txt,./descargas

Requisitos:
- Módulo 'requests' instalado en el entorno Python donde se ejecute este script.



"""

import requests
import os


def descargar_archivo_directo(id_archivo, directorio_destino, archivo_destino):
    """
    Descarga un archivo directamente desde Google Drive y lo guarda localmente.

    Parámetros:
    - id_archivo (str): ID del archivo en Google Drive.
    - directorio_destino (str): Ruta del directorio local donde se guardará el archivo.
    - archivo_destino (str): Nombre deseado para el archivo en local

    Devuelve:
    - archivo_destino (str): Nombre del archivo guardado.
    - ruta_completa (str): Ruta completa del archivo guardado.
    """
    # Construye la URL de descarga directa utilizando el ID del archivo
    url = f"https://drive.google.com/uc?export=download&id={id_archivo}"

    # Realiza la petición HTTP GET para descargar el archivo
    respuesta = requests.get(url, allow_redirects=True)

    # Comprueba que el directorio destino existe, si no, lo crea
    os.makedirs(directorio_destino, exist_ok=True)

    # Construye la ruta completa donde se guardará el archivo en local
    ruta_completa = os.path.join(directorio_destino, archivo_destino)

    # Guarda el contenido del archivo descargado en local
    with open(ruta_completa, 'wb') as archivo:
        archivo.write(respuesta.content)


    return archivo_destino, ruta_completa


def procesar_archivo_info(ruta_archivo_info):
    """
    Procesa un archivo de texto que contiene información sobre los archivos a descargar.

    Parámetros:
    - ruta_archivo_info (str): Ruta del archivo de texto que contiene los IDs de Google Drive,
                               los nombres de los archivos locales y las rutas locales.

    Devuelve:
    - Una lista de tuplas con el ID de Google Drive, el nombre local del archivo, y la ruta local.
    """
    archivos_info = []
    with open(ruta_archivo_info, 'r') as archivo:
        for linea in archivo:
            id_archivo, nombre_archivo, directorio_destino = linea.strip().split(',')
            archivos_info.append((id_archivo, nombre_archivo, directorio_destino))
    return archivos_info


In [54]:
# Ruta al archivo que contiene la información de los archivos a descargar
ruta_archivo_info = "..\info_archivos_GDrive_solo_entr.txt"

# Obtenemos la lista con la info de los archivos del fichero
archivos_a_descargar = procesar_archivo_info(ruta_archivo_info)

# Descargamos cada archivo de la lista
for id_archivo, nombre_archivo, directorio_destino in archivos_a_descargar:
    nombre_archivo_descargado, ruta_archivo_guardado = descargar_archivo_directo(id_archivo, directorio_destino, nombre_archivo)
    print(f"Archivo {nombre_archivo_descargado} guardado en: {ruta_archivo_guardado}")

Archivo datos_preparados_solo_entr.parquet guardado en: ../Downloads\datos_preparados_solo_entr.parquet


In [55]:
df = pd.read_parquet(ruta_archivo_guardado, engine='pyarrow')
df.describe()

,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Tackles,...,nationality_The Gambia,nationality_Türkiye,nationality_Ukraine,nationality_United States,nationality_Uruguay,nationality_Venezuela,nationality_Wales,nationality_Zambia,nationality_Zimbabwe,marketValue
count,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,...,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2.251000e+03
mean,1271.695691,1.564638,1.063527,3.069303,0.168814,0.629276,76.128387,0.946246,0.622834,0.882719,...,0.000444,0.002665,0.001333,0.003554,0.028432,0.003998,0.000888,0.000444,0.000444,8.160329e+06
std,969.241058,2.906800,1.737932,2.958017,0.430942,0.604061,13.308598,0.962542,1.244505,0.693640,...,0.021077,0.051571,0.036490,0.059522,0.166240,0.063119,0.029801,0.021077,0.021077,1.188624e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000e+04
25%,360.500000,0.000000,0.000000,1.000000,0.000000,0.200000,70.950000,0.300000,0.000000,0.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000e+06
50%,1157.000000,0.000000,0.000000,2.000000,0.000000,0.500000,78.300000,0.700000,0.000000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000e+06
75%,2040.500000,2.000000,2.000000,5.000000,0.000000,0.900000,84.200000,1.300000,1.000000,1.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.500000e+06
max,3420.000000,27.000000,16.000000,15.000000,3.000000,4.500000,100.000000,8.700000,13.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.500000e+07


In [56]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# Partición de datos
RANDOM_STATE = 83
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)

In [57]:
# Establecer la URI de la base de datos SQLite
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# Configuración de MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("arboles")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1713174288421, experiment_id='1', last_update_time=1713174288421, lifecycle_stage='active', name='arboles', tags={}>

# Evaluación Por Pos Sobre General

In [58]:
df_original = df.copy()
# Crear y entrenar el modelo de árbol de decisión
clf = DecisionTreeRegressor(criterion="absolute_error", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Predecir con el modelo entrenado
y_pred_rf = clf.predict(X_test)

# Calcular el error
error = mean_absolute_error(y_test, y_pred_rf)
print("Arbol de decision:", error)

# Crear DataFrame de predicciones vs. reales
pred_vs_real = pd.DataFrame({'y_test': y_test, 'y_pred_rf': y_pred_rf, 'dif': abs(y_test-y_pred_rf)})
posiciones = ['Left-Back', 'Centre-Back', 'Defensive Midfield',
       'Centre-Forward','Second Striker','Central Midfield', 'Left Winger',
       'Attacking Midfield', 'Right-Back','Right Winger',
       'Left Midfield', 'Goalkeeper']
# Mostrar el DataFrame
print(pred_vs_real)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="COMPORTAMIENTO POSICIONES EN MODELO COMPLETO") as run:
    # Registrar los hiperparámetros
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    
    # Registrar la métrica de error
    mlflow.log_metric("mean_absolute_error", error)
    
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar el DataFrame como un artefacto
    pred_vs_real.to_csv("pred_vs_real.csv", index=False)
    mlflow.log_artifact("pred_vs_real.csv")
    
    # Calcular medias por grupos de posiciones
    li = []
    for p in posiciones:
        solo_posicion = df_original.loc[pred_vs_real.index][df_original.loc[pred_vs_real.index, 'position_' + p] == 1]
        mean_dif = np.mean(pred_vs_real.loc[solo_posicion.index]['dif'])
        li.append(mean_dif)

    mean_defenders = np.mean(li[0:3])
    mean_strikers = np.mean(li[3:5])
    mean_midfielders = np.mean(li[5:11])
    mean_goalkeepers = li[11]

    print('Defensas', mean_defenders)
    print('Delanteros', mean_strikers)
    print('Medios', mean_midfielders)
    print('Porteros', mean_goalkeepers)

    # Guardar las métricas específicas en MLflow
    mlflow.log_metric("mean_error_defenders", mean_defenders)
    mlflow.log_metric("mean_error_strikers", mean_strikers)
    mlflow.log_metric("mean_error_midfielders", mean_midfielders)
    mlflow.log_metric("mean_error_goalkeepers", mean_goalkeepers)

Arbol de decision: 3293565.0887573967
          y_test   y_pred_rf        dif
893    1000000.0   1000000.0        0.0
827    1200000.0   1200000.0        0.0
149    8000000.0   6000000.0  2000000.0
872    3000000.0   1200000.0  1800000.0
1173  14000000.0  10000000.0  4000000.0
...          ...         ...        ...
692     150000.0    200000.0    50000.0
2091   1500000.0    800000.0   700000.0
1103   6000000.0   2500000.0  3500000.0
1589   4000000.0   9500000.0  5500000.0
605    5000000.0   3000000.0  2000000.0

[676 rows x 3 columns]


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Defensas 3308995.7264957265
Delanteros 1264224.1379310344
Medios 4033686.1604711376
Porteros 2029032.2580645161


# Con Hiperparámetros

In [60]:
clf = DecisionTreeRegressor(criterion="absolute_error", max_depth = 30, max_features=None, max_leaf_nodes= 80, min_samples_leaf=5, min_samples_split=12, splitter="random", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

DecisionTreeRegressor(criterion='absolute_error', max_depth=30,
                      max_leaf_nodes=80, min_samples_leaf=5,
                      min_samples_split=12, random_state=83, splitter='random')

In [61]:
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

# Predecir con el modelo entrenado
y_pred_rf = clf.predict(X_test)

# Calcular el error
error = mean_absolute_error(y_test, y_pred_rf)
print("Arbol de decision:", error)

# Calcular el R^2 del modelo
r2 = r2_score(y_test, y_pred_rf)

# Calcular el número de predictores y el tamaño de la muestra
n = len(y_test)
k = X_test.shape[1]  # Número de columnas de X_test

# Calcular el R^2 ajustado
r2_adj = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

print("R^2 ajustado:", r2_adj)

# Crear DataFrame de predicciones vs. reales
pred_vs_real = pd.DataFrame({'y_test': y_test, 'y_pred_rf': y_pred_rf, 'dif': abs(y_test-y_pred_rf)})

# Mostrar el DataFrame
print(pred_vs_real)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="FINAL MODEL TREE") as run:
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    mlflow.log_param("max_depth", 30)
    mlflow.log_param("max_features", None)
    mlflow.log_param("max_leaf_nodes", 80)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("min_samples_split", 12)
    mlflow.log_param("splitter", "random")
    # Registrar la métrica de error
    mlflow.log_metric("mean_absolute_error", error)
    mlflow.log_metric("r2_ajustado", r2_adj)
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar el DataFrame como un artefacto
    pred_vs_real.to_csv("pred_vs_real.csv", index=False)
    mlflow.log_artifact("pred_vs_real.csv")


Arbol de decision: 3174704.1420118343
R^2 ajustado: 0.6501141195641189
          y_test  y_pred_rf         dif
893    1000000.0  1500000.0    500000.0
827    1200000.0  1200000.0         0.0
149    8000000.0  5000000.0   3000000.0
872    3000000.0   850000.0   2150000.0
1173  14000000.0  4000000.0  10000000.0
...          ...        ...         ...
692     150000.0   400000.0    250000.0
2091   1500000.0   850000.0    650000.0
1103   6000000.0  8000000.0   2000000.0
1589   4000000.0  5000000.0   1000000.0
605    5000000.0  5000000.0         0.0

[676 rows x 3 columns]


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Sin hiperparámetros

In [46]:
clf = DecisionTreeRegressor(criterion="absolute_error", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

DecisionTreeRegressor(criterion='absolute_error', random_state=83)

In [48]:
# Predecir con el modelo entrenado
y_pred_rf = clf.predict(X_test)

# Calcular el error
error = mean_absolute_error(y_test, y_pred_rf)
print("Arbol de decision:", error)

# Crear DataFrame de predicciones vs. reales
pred_vs_real = pd.DataFrame({'y_test': y_test, 'y_pred_rf': y_pred_rf, 'dif': abs(y_test-y_pred_rf)})

# Mostrar el DataFrame
print(pred_vs_real)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="sin hiperparámetros evaluación") as run:
    # Registrar la métrica de error
    mlflow.log_metric("mean_absolute_error", error)
    
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
        # Calcular el R^2 del modelo
    r2 = r2_score(y_test, y_pred_rf)
    
    # Calcular el número de predictores y el tamaño de la muestra
    n = len(y_test)
    k = X_test.shape[1]  # Número de columnas de X_test
    
    # Calcular el R^2 ajustado
    r2_adj = 1 - ((1 - r2) * (n - 1) / (n - k - 1))
    
    mlflow.log_metric("r2_ajustado", r2_adj)
    # Guardar el modelo
    # Guardar el DataFrame como un artefacto
    pred_vs_real.to_csv("pred_vs_real.csv", index=False)
    mlflow.log_artifact("pred_vs_real.csv")

Arbol de decision: 3293565.0887573967
          y_test   y_pred_rf        dif
893    1000000.0   1000000.0        0.0
827    1200000.0   1200000.0        0.0
149    8000000.0   6000000.0  2000000.0
872    3000000.0   1200000.0  1800000.0
1173  14000000.0  10000000.0  4000000.0
...          ...         ...        ...
692     150000.0    200000.0    50000.0
2091   1500000.0    800000.0   700000.0
1103   6000000.0   2500000.0  3500000.0
1589   4000000.0   9500000.0  5500000.0
605    5000000.0   3000000.0  2000000.0

[676 rows x 3 columns]


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
